In [ ]:
"""
<Open-Api 방식>
 - 특정 기관에서 데이터를 제공하는 방식으로,
 - 웹URL을 이용해서 제공하는 방식(정확하게는 웹프로토콜을 이용해서 제공)
  -> 프로토콜이라 함은 : 규약, 규칙이라는 의미입니다.
 - 데이터를 요청하는 사람(수집자)과 / 데이터를 제공하는 기관(제공자)이 있습니다.
 - 수집자는 제공자가 제공하는 URL을 사용해야 하며,
  -> 제공자가 정의한 key : value의 값들을 받아와서 사용해야 합니다.
 - 제공자는 아무에게나 데이터를 제공하지 않으며, 승인된 수집자에게만 데이터를 제공합니다.
  -> 승입여부 : 제공자가 발급한 API-Key를 수집자가 전달 받아서 요청 시 사용합니다.

 <중요 용어>
 - 수집자 == 요청자(클라이언트, client) == request
 - 제공자 == 응답자(서버, server) == response
"""

In [ ]:
"""
 - 공공데이터포털 접속 : www.data.go.kr
 - 회원가입 되어 있어야 합니다. (로그인 필요)
 - 검색어에 '한국공학공사 전국공항 주차장 혼잡도'로 검색
 - [활용신청] 버튼 클릭 
     -> 활용 신청 시 활용목적을 작성할 때는 '학술/연구'로...
"""

## 사용할 라이브러리 정의

In [1]:
### 데이터 처리
import pandas as pd

### 웹 프로토콜을 통해서 요청 URL을 전송하고 결과를 받아오는 라이브러리
import requests

### 전달받은 태그 코드의 데이터를 검색 추출하기 위한 라이브러리
from bs4 import BeautifulSoup

### 전달받은 태그 코드를 인식하지 못하는 경우를 대비해서 라이브러리 추가만 해놓기
# - 설치 필요 : pip install lxml
# - 설치 후 커널 재시작해야 합니다.
from openpyxl.workbook import Workbook

## 요청 처리를 위한 변수 정의하기

In [2]:
### API 인증키 변수정의
# api_key = '742HGd21WfbEf7CQo47sG4r%2B5bKm38NSkuyUQthDu1%2BjhyylvgAN0BgU7nB%2BNgp2l%2Bc27O4TxWRP3koINEP4%2Fw%3D%3D'
api_key='PVdeh44WlF%2BHj6bGzfI2BXUldlea7Ggo%2FjjXvhbHCMNNFotchb2byGTopt4uAFS07RQzqmeU3M2Y8HcMmlYW9w%3D%3D'

### 요청에 사용할 공항코드 변수 정의
schAirportCode_list = ['GMP','PUS', 'CJU', 'TAE', 'KWJ',
                       'RSU', 'USN', 'KUV', 'WJU', 'CJJ']

### 한페이지 행의 수
numOfRows = 10

### 페이지 번호
pageNo = 1

In [3]:
### 요청 URL 완성하기
api_url = 'http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT'
api_url += f'?schAirportCode={schAirportCode_list[0]}'
api_url += f'&serviceKey={api_key}&numOfRows={numOfRows}&pageNo={pageNo}'
api_url

'http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT?schAirportCode=GMP&serviceKey=PVdeh44WlF%2BHj6bGzfI2BXUldlea7Ggo%2FjjXvhbHCMNNFotchb2byGTopt4uAFS07RQzqmeU3M2Y8HcMmlYW9w%3D%3D&numOfRows=10&pageNo=1'

## 요청 및 응답 받아오기

In [4]:
### requests 라이브러리를 이용해서 서버로 url을 넘겨주기(요청)
rs_data = requests.get(api_url)
rs_data
### Response [200] : Response 클래스 객체에 의해 잘 전송받았다는 코드
# - API-Key 발급 후 즉시 데이터는 받아오지 못하는 경우 있음(인증에 시간 걸림)

<Response [200]>

In [5]:
### 응답 받은 데이터 확인하기
rs_text = rs_data.text
rs_text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><airportEng>GIMPO INTERNATIONAL AIRPORT</airportEng><airportKor>김포국제공항</airportKor><parkingAirportCodeName>국내선 제1주차장</parkingAirportCodeName><parkingCongestion>원활</parkingCongestion><parkingCongestionDegree>85.65%</parkingCongestionDegree><parkingOccupiedSpace>1952</parkingOccupiedSpace><parkingTotalSpace>2279</parkingTotalSpace><sysGetdate>2025-02-21</sysGetdate><sysGettime>08:58:03</sysGettime></item><item><airportEng>GIMPO INTERNATIONAL AIRPORT</airportEng><airportKor>김포국제공항</airportKor><parkingAirportCodeName>국내선 제2주차장</parkingAirportCodeName><parkingCongestion>원활</parkingCongestion><parkingCongestionDegree>50.14%</parkingCongestionDegree><parkingOccupiedSpace>869</parkingOccupiedSpace><parkingTotalSpace>1733</parkingTotalSpace><sysGetdate>2025-02-21</sysGetdate><sysGettime>08:58:03</sysGettime></item><item><airportEn

In [6]:
### 문자열 타입의 데이터에 -> 태그(xml) 포맷이라는 의미 부여시키기
# - 의미 부여를 -> 파싱(변환, parsing) 처리라고 합니다.
# - BeautifulSoup() 라이브러리를 사용합니다.
rs_xml = BeautifulSoup(rs_text, 'xml')
rs_xml

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><airportEng>GIMPO INTERNATIONAL AIRPORT</airportEng><airportKor>김포국제공항</airportKor><parkingAirportCodeName>국내선 제1주차장</parkingAirportCodeName><parkingCongestion>원활</parkingCongestion><parkingCongestionDegree>85.65%</parkingCongestionDegree><parkingOccupiedSpace>1952</parkingOccupiedSpace><parkingTotalSpace>2279</parkingTotalSpace><sysGetdate>2025-02-21</sysGetdate><sysGettime>08:58:03</sysGettime></item><item><airportEng>GIMPO INTERNATIONAL AIRPORT</airportEng><airportKor>김포국제공항</airportKor><parkingAirportCodeName>국내선 제2주차장</parkingAirportCodeName><parkingCongestion>원활</parkingCongestion><parkingCongestionDegree>50.14%</parkingCongestionDegree><parkingOccupiedSpace>869</parkingOccupiedSpace><parkingTotalSpace>1733</parkingTotalSpace><sysGetdate>2025-02-21</sysGetdate><sysGettime>08:58:03</sysGettime></item><item><airportEng>GIMPO INTERNATI

In [7]:
### 태그 이름 : key
### 태그와 태그 사이의 텍스트 : value
# - find_all(찾고자하는 태그이름 - key) : 해당 태그이름이 있는 모든 태그를 리스트 타입으로 변환
rs_xml.find_all('airportKor')

[<airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>]

In [8]:
# - find(찾고자하는 태그이름 - key) : 해당 태그이름이 있는 모든 태그 중에서 첫번째 태그만 반환 (xml 타입)
rs_xml.find('airportKor')

<airportKor>김포국제공항</airportKor>

In [9]:
### 태그와 태그사잉의 값 추출하기
# - xml 타입인 경우만 가능
# - 문자열로 반환함
rs_xml.find("airportKor").text

'김포국제공항'

In [10]:
rs_xml.find_all("airportKor")[0].text

'김포국제공항'

In [11]:
### 주차장 혼잡률에 대한 모든 태그 정보 가지고 오기...
rs_xml.find_all('parkingCongestionDegree')[0].text

'85.65%'

In [12]:
### 혼잡도
print(f"혼잡도 : {rs_xml.find_all('parkingCongestion')[0].text}")
### 주차장이름
print(f"주차장이름 : {rs_xml.find_all('parkingAirportCodeName')[0].text}")
### 총주차면수
print(f"총주차면수 : {rs_xml.find_all('parkingTotalSpace')[0].text}")
### 입고된차량수
print(f"입고된차량수 : {rs_xml.find_all('parkingOccupiedSpace')[0].text}")
### 수집 기준년월일
print(f"수집 기준년월일 : {rs_xml.find_all('sysGetdate')[0].text}")
### 수집 기준시간
(f"수집 기준시간 : {rs_xml.find_all('sysGettime')[0].text}")
### 주차 가능한 여유공간 수
tot = int(rs_xml.find_all('parkingTotalSpace')[0].text)
occ = int(rs_xml.find_all('parkingOccupiedSpace')[0].text)
print(f"주차 가능한 여유공간 수: {tot - occ}")

혼잡도 : 원활
주차장이름 : 국내선 제1주차장
총주차면수 : 2279
입고된차량수 : 1952
수집 기준년월일 : 2025-02-21
주차 가능한 여유공간 수: 327


In [ ]:
"""
김포 국제공항에 대한 전체 주차장 정보를 출력해 주세요..
출력 결과는 아래와 같이..
[공항명] : 김포국제공항
[기준 년월일 시분초] : 0000-00-00 00:00:00
--------------------------------------------
[주차장명] : 국내선 제1주차장
 총주차면수 : 111
 ...
 주차장 정보 모두
 -------------------------------------------
[주차장명] : 국내선 제2주차장
 총주차면수 : 222
 ...
 주차장 정보 모두
 --------------------------------------------
 

"""

In [13]:
print(f"[공항명] : {rs_xml.find_all('airportKor')[0].text}")
### 수집 기준년월일
print(f"[기준 년월일 시분초] : {rs_xml.find_all('sysGetdate')[0].text} {rs_xml.find_all('sysGettime')[0].text}")
print("--------------------------------------------------")
### 주차장이름
print(f"[주차장명] : {rs_xml.find_all('parkingAirportCodeName')[0].text}")
### 총주차면수
print(f" - 총주차면수 : {rs_xml.find_all('parkingTotalSpace')[0].text}")
### 입고된차량수
print(f" - 입고된차량수 : {rs_xml.find_all('parkingOccupiedSpace')[0].text}")
### 주차 가능한 여유공간 수
tot = int(rs_xml.find_all('parkingTotalSpace')[0].text)
occ = int(rs_xml.find_all('parkingOccupiedSpace')[0].text)
print(f" - 주차 가능한 여유공간 수: {tot - occ}")

[공항명] : 김포국제공항
[기준 년월일 시분초] : 2025-02-21 08:58:03
--------------------------------------------------
[주차장명] : 국내선 제1주차장
 - 총주차면수 : 2279
 - 입고된차량수 : 1952
 - 주차 가능한 여유공간 수: 327


In [14]:
for i in range(len(rs_xml.find_all('airportKor'))):
    print(f"[공항명] : {rs_xml.find_all('airportKor')[i].text}")
    ### 수집 기준년월일
    print(f"[기준 년월일 시분초] : {rs_xml.find_all('sysGetdate')[i].text} {rs_xml.find_all('sysGettime')[i].text}")
    print("------------------------------------------")
    ### 주차장이름
    print(f"[주차장명] : {rs_xml.find_all('parkingAirportCodeName')[i].text}")
    ### 총주차면수
    print(f" - 총 주차면 수 : {rs_xml.find_all('parkingTotalSpace')[i].text}")
    ### 입고된차량수
    print(f" - 입고된 차량 수 : {rs_xml.find_all('parkingOccupiedSpace')[i].text}")
    ### 주차 가능한 여유공간 수
    tot = int(rs_xml.find_all('parkingTotalSpace')[i].text)
    occ = int(rs_xml.find_all('parkingOccupiedSpace')[i].text)
    print(f" - 주차 가능한 여유공간 수: {tot - occ}")
    print("------------------------------------------")
    print("")

[공항명] : 김포국제공항
[기준 년월일 시분초] : 2025-02-21 08:58:03
------------------------------------------
[주차장명] : 국내선 제1주차장
 - 총 주차면 수 : 2279
 - 입고된 차량 수 : 1952
 - 주차 가능한 여유공간 수: 327
------------------------------------------

[공항명] : 김포국제공항
[기준 년월일 시분초] : 2025-02-21 08:58:03
------------------------------------------
[주차장명] : 국내선 제2주차장
 - 총 주차면 수 : 1733
 - 입고된 차량 수 : 869
 - 주차 가능한 여유공간 수: 864
------------------------------------------

[공항명] : 김포국제공항
[기준 년월일 시분초] : 2025-02-21 08:58:03
------------------------------------------
[주차장명] : 국제선 주차빌딩
 - 총 주차면 수 : 567
 - 입고된 차량 수 : 487
 - 주차 가능한 여유공간 수: 80
------------------------------------------

[공항명] : 김포국제공항
[기준 년월일 시분초] : 2025-02-21 08:58:03
------------------------------------------
[주차장명] : 국제선 지하
 - 총 주차면 수 : 599
 - 입고된 차량 수 : 508
 - 주차 가능한 여유공간 수: 91
------------------------------------------

[공항명] : 김포국제공항
[기준 년월일 시분초] : 2025-02-21 08:58:03
------------------------------------------
[주차장명] : 화물청사
 - 총 주차면 수 : 737
 - 입고된 차량 수 : 737
 - 주차 가능한